<h2 style="text-align: center; background-color: #4d648d; font-family:Arial; color: white; padding: 13px; line-height: 1; border-radius:10px"> 03_03 Analyse de sentiments</h2>

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, token="XXXXXXXXXXXXXXXXXXXXXXx")
tokenizer.pad_token = tokenizer.eos_token  # pour éviter l'erreur
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", token="XXXXXXXXXXXXXXXXXXXXXXx")

def sentiment_llm(text):
    prompt = f"Quel est le sentiment de ce texte ? Réponds par 'positif', 'neutre' ou 'négatif'.\n\nTexte : {text}\nRéponse :"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=10)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response.split("Réponse :")[-1].strip()

print(sentiment_llm("J'adore ce produit, il est vraiment génial !"))


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

#model_name = "cardiffnlp/twitter-roberta-base-sentiment"
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        scores = F.softmax(outputs.logits, dim=1).squeeze()
        score = torch.argmax(scores).item()
        return {
            "sentiment": score + 1,  # souvent 0 = 1 étoile
            "scores": scores.tolist()
        }

# 🧪 Exemple
result = sentiment_analysis("J'adore ce produit, il est vraiment génial !")
print(result)

In [ ]:
result = sentiment_analysis("Le produit est conforme à la photo.J'ai pas la même couette donc ça donne pas le même rendu que sur la photo . Après c'est de la qualité basique... vous attendez pas à un truc de fou.")
print(result)